<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema5_8_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   


##5.8 Cómo ajustar (fine-tuning) un transformer

En este ejercicio, aprenderemos a ajustar (fine-tuning) el modelo pre-entrenado BERT para la tarea clasificación de textos.

**Fine-tuning** consiste en utilizar un modelo (ya pre-entrenado) y  y entrenarlo sobre un dataset para una tarea concreta, como por ejemplo, clasificación de textos o NER. Las principales ventajas de este proceso en comparación a entrenar un modelo desde cero:
- menor coste computacional (vectores ya están pre-entrenados)
- reducen el tiempo y esfuerzo porque puedes utlizar modelos sin tener que entrenar desde cero.

En nuestro caso, ajustaremos el transformer BERT base (versión uncased) a la tarea de clasificación de textos sobre el dataset de rotten_tomatoes (formado por revisiones sobre películas, clasificadas con dos polaridades: positiva y negativa).

En este ejercicio, usaremos **Pytorch** para entrenar nuestro modelo, aunque también es posible utilizar tensorflow (https://huggingface.co/docs/transformers/training#loading-data-for-keras).


### Instalación de librerías

In [3]:
!pip install -q transformers[torch] datasets


## Cargamos el dataset



In [4]:
from datasets import load_dataset
dict_dataset = load_dataset("rotten_tomatoes")
dict_dataset

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

Las labels son:

In [5]:
LABELS = dict_dataset['train'].features['label'].names
NUM_LABELS = len(LABELS)
print('LABELS:', LABELS, 'num_labels:', NUM_LABELS )

LABELS: ['neg', 'pos'] num_labels: 2


## Tokenización

Necesitamos preparar los textos para que tengan el formato de entrada que el transformer necesita. Debemos cargar el tokenizador asociado a BERT base.


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

BERT admite 512 tokens como tamaño de la entrada.
¿Cuál es el tamaño (número de tokens) en nuestro dataset?

In [7]:
MAX_LENGTH= max([len(tokenizer(text).input_ids) for text in dict_dataset['train']['text']])
print("Tamaño máximo", MAX_LENGTH)

Tamaño máximo 78




Podríamos estudiar cómo es la distribución de los tamaños de los textos en esta colección. Sin embargo, como su tamaño máximo es 78, y este tamaño es mucho menor que el admitido por BERT (512 tokens), en este ejemplo, hemos decidido usar 78 como la longitud máxima.

No es necesario definir **truncation** a True, porque no será necesario recortar ningún texto.

La tokenizacion por lotes (batch) es más rápida.
Definimos una función que nos permite tokenizar por lotes.



In [11]:
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length",max_length=MAX_LENGTH)

También podríamos haber inicializado el parámetro **padding = longest**. En este caso, todos los textos del mismo lote tendrían la longitud del texto más largo en dicho lote.


El método **map** nos permite aplicar la función anterior a todos los splits:

In [12]:
encoded_data = dict_dataset.map(tokenize, batched=True)
encoded_data

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1066
    })
})

Podemos comprobar que todos los textos tienen el mismo tamaño

In [13]:
import random
for i in range(10):
    index = random.randint(0,encoded_data['train'].num_rows)
    print('text:', index, ' len:', len(encoded_data['train'][index]['input_ids']))


text: 4034  len: 78
text: 3218  len: 78
text: 2796  len: 78
text: 6703  len: 78
text: 1280  len: 78
text: 6903  len: 78
text: 4006  len: 78
text: 4669  len: 78
text: 1283  len: 78
text: 6603  len: 78


## Fine-tuning de un modelo pre-entrenado


El primer paso será cargar el modelo que vamos a ajustar. Como hemos dicho antes, será el modelo BERT base en su versión uncased. La clase **AutoModelForSequenceClassification** nos permite cargar dicho modelo ya extendido para la tarea de clasificación de secuencias. En concreto, este modelo extendido tiene una última capa softmax que obtiene una probabilidad por cada clase. En nuestro caso, es un problema de clasificación binaria. Además, del modelo es necesario, indicar el número de clases (labels).

In [14]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


La celda anterior nos avisa que algunos parámetros no han sido ajustados. Esto es completamente normal.

#### Hiperparámetros


Necesitamos definir los hiperparámetros que se emplearán durante el proceso de entrenamiento. Algunos de estos hiperpárametros son el número de epochs para el entrenamiento, el tamaño del lote (batch), el ratio de aprendizaje (learning rate), etc.  Puedes experimentar con estos hiperparámetros para encontrar la configuración óptima para tu tarea.

En nuestro caso, trabajaremos con los hipérparametros por defecto.

Para ello vamos a crear un objeto de la clase **TrainingArguments**, que incluye todos los hiperparámetros (ya inicializados con valores por defecto) que puedes ajustar.


In [16]:
from transformers import TrainingArguments
args = TrainingArguments(output_dir="./outputs")
args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_model

En la siguiente celda, vamos a modificar el tamaño del lote para entrenamiento y para validación:

In [18]:
args.per_device_train_batch_size = 32
args.per_device_eval_batch_size = 32


También vamos a indicar la estrategia para entrenar el modelo, que puede ser epoch o step. Los pasos (steps) se refieren a la cantidad de lotes procesados por el modelo durante el entrenamiento, mientras que las épocas se refieren a un paso completo por todo el conjunto de datos de entrenamiento.




In [19]:
args.evaluation_strategy="epoch"


#### Métricas


También tenemos que definir el conjunto de métricas que se utilizaran para evaluar el modelo sobre el conjunto de validación. Este conjunto de métricas depende de cada tarea. En el caso de la clasificación de textos, además del accuracy, es interesante conocer la precisión, recall y f1.


In [20]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):

    y_true = pred.label_ids                 # son las labels reales
    y_pred = pred.predictions.argmax(-1)    # son las predicciones


    acc = accuracy_score(y_true, y_pred)

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


### Trainer


Ya estamos listos para entrenar el modelo. Para ello, Pytorch ya nos proporciona una clase **Trainer**,  que está optimizada para entrenar transformers, y que además nos va a ahorrar mucho trabajo (no será necesario escribir el ciclo de entrenamiento por epochs y calcular las métricas sobre el conjunto de validación).

Para crear este objeto Trainer, deberemos pasarle el modelo, los argumentos, la función para calcular las métricas, y el conjunto de entrenamiento y validación.

In [21]:
from transformers import Trainer

trainer = Trainer(
    model = model,            # modelo que será ajustado
    train_dataset = encoded_data['train'], # conjunto training
    eval_dataset = encoded_data['validation'],   # conjunto de validación

    args = args,     # hiperparámetros
    compute_metrics=compute_metrics,    # función para computar las métricas
)

Para entrenar simplemente tenemos que invocar al método train (tardará unos minutos):

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.317526,0.858349,0.857884,0.863105,0.858349
2,0.283200,0.366538,0.865854,0.865655,0.868031,0.865854
3,0.283200,0.574441,0.863977,0.863963,0.864133,0.863977


TrainOutput(global_step=801, training_loss=0.20540292908934024, metrics={'train_runtime': 359.963, 'train_samples_per_second': 71.091, 'train_steps_per_second': 2.225, 'total_flos': 1025732282655600.0, 'train_loss': 0.20540292908934024, 'epoch': 3.0})

En la celda, podemos ver como los resultados han variado muy poco de una epoch a otra. Incluso el error sobre el conjunto de validación aumenta.
Sería interesante entrenar con la estrategia de steps, o incluso probar con otros argumentos, para ver su efecto.


Una vez finalizado el entrenamiento, es posible evaluar el modelo final sobre el conjunto de datos de validación. Las métricas obtenidas son:

In [24]:
trainer.evaluate()

{'eval_loss': 0.5744408965110779,
 'eval_accuracy': 0.8639774859287055,
 'eval_f1': 0.8639630006116693,
 'eval_precision': 0.8641325783186837,
 'eval_recall': 0.8639774859287055,
 'eval_runtime': 5.3259,
 'eval_samples_per_second': 200.154,
 'eval_steps_per_second': 6.384,
 'epoch': 3.0}

Sobre el conjunto de validación, la f1 es de un 86.3%.

## Evaluación

Por último, vamos a utilizar el modelo para predecir las clases de los textos del conjunto test (estos textos que no han sido utilizados durane el entrenamiento).

La siguiente función recibe un texto y devuelve la clase inferida por el modelo. La función codifica el texto usando el tokenizador y el modelo es aplicado sobre esta codificación. Sobre la salida del modelo, aplicaremos una función softmax, que calcule la probabilidad de cada clase. Finalmente, devolvemos la clase con mayor probabilidad (usaremos la función **argmax**).


In [25]:
def get_prediction(text):
    # prepara el texto, aplicamos la misma tokenización que la utilizada en el training
    inputs = tokenizer(text, padding="max_length", max_length=MAX_LENGTH, truncation= True, return_tensors="pt").to("cuda")

    # aplicamos el modelo
    pred = model(**inputs).logits

    # obtenemos la probabilidad para cada clase
    probs = pred.softmax(1)
    # devolvemos la mayor
    return probs.argmax().item()

Ahora procesamos todos los textos del conjunto test con esa función para obtener qué clase infiere el modelo para cada texto. Las predicciones son almacenadas en la lista y_pred. La lista y_true contiene las labels originales en el conjunto test.


In [26]:
y_pred=[get_prediction(text) for text in dict_dataset['test']['text']]
y_true = dict_dataset['test']['label']


La función **classification_report** de sklearn nos permite fácilmente calcular las métricas precisión, recall y f1 para cada clase, además del acuracy y las macros.


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=LABELS))

              precision    recall  f1-score   support

         neg       0.85      0.86      0.86       533
         pos       0.86      0.85      0.86       533

    accuracy                           0.86      1066
   macro avg       0.86      0.86      0.86      1066
weighted avg       0.86      0.86      0.86      1066



Los resultados son similares al conjunto de evaluación. La f1 de las clases positivas y negativas es 86% (tiene sentido porque están balanceadas). La macro y el acuracy también 86%.

## Inferir
¿Cómo usamos el modelo para inferir sobre un texto nuevo?
Por ejemplo, vamos a tomar el texto con índice 1 del conjunto test:

In [28]:
text = dict_dataset['test'][1]['text']
label = dict_dataset['test'][1]['label']
print(text, ' LABEL:', label)
print()


consistently clever and suspenseful .  LABEL: 1



Tokenizamos el texto para codificarlo en el mismo formato utilizado para el training y luego aplicamos el modelo sobre su codificación:

In [29]:
# tokenizamos: prepara el texto, aplicamos la misma tokenización que la utilizada en el training; además añadimos truncation
inputs = tokenizer(text, padding="max_length", max_length=MAX_LENGTH, truncation= True, return_tensors="pt").to("cuda")

# aplicamos el modelo
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-3.8632,  3.7808]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [30]:
print("Predicciones generadas por el modelo:", outputs.logits)

# Pasamos a probabilidades usando softmax
probs =  outputs.logits.softmax(1)
print("Probabilidad para cada clase:", probs)
print()


Predicciones generadas por el modelo: tensor([[-3.8632,  3.7808]], device='cuda:0', grad_fn=<AddmmBackward0>)
Probabilidad para cada clase: tensor([[4.7868e-04, 9.9952e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)



In [31]:
print('Qué clase es la que tiene mayor probabilidad?:', probs.argmax())
# probs.argmax() devuelve un tensor. Debemos tomar únicamente el campo
print('Qué clase es la que tiene mayor probabilidad?:', probs.argmax().item())


Qué clase es la que tiene mayor probabilidad?: tensor(1, device='cuda:0')
Qué clase es la que tiene mayor probabilidad?: 1


El modelo acierta para ese ejemplo!!!

Ejecuta el siguiente código varias veces. Podrás observar para qué ejemplos el modelo es capaz de inferir la clase correcta, y para qué ejemplos falla.


In [35]:
import random

index = random.randint(0, dict_dataset['test'].num_rows)
text = dict_dataset['test'][index]['text']
label = dict_dataset['test'][index]['label']
print(text, ' LABEL:', label)
print()

# tokenizamos: prepara el texto, aplicamos la misma tokenización que la utilizada en el training; además añadimos truncation
inputs = tokenizer(text, padding="max_length", max_length=MAX_LENGTH, truncation= True, return_tensors="pt").to("cuda")

# aplicamos el modelo
outputs = model(**inputs)
outputs

print("Predicciones generadas por el modelo:", outputs.logits)

# Pasamos a probabilidades usando softmax
probs =  outputs.logits.softmax(1)
print("Probabilidad para cada clase:", probs)
print()

print('Clase inferida:', probs.argmax().item())
print('Clase en el conjunto test:', label)



this is popcorn movie fun with equal doses of action , cheese , ham and cheek ( as well as a serious debt to the road warrior ) , but it feels like unrealized potential  LABEL: 1

Predicciones generadas por el modelo: tensor([[ 2.5660, -2.6890]], device='cuda:0', grad_fn=<AddmmBackward0>)
Probabilidad para cada clase: tensor([[0.9948, 0.0052]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

Clase inferida: 0
Clase en el conjunto test: 1


Puedes encontrar más información sobre el proceso de fine-tuning en el siguiente enlace
https://huggingface.co/docs/transformers/training.

Te animo a que explores otros modelos de Hugging Face (https://huggingface.co/models), y trates de ajustarlos para esta tarea. Por ejemplo, algunos poddrían ser:

- bert-base-cased
- distilbert-base-uncased
- albert-base-v2
- xlm-roberta-base

Esto te permitirá comparar los resultados de los modelos y ver qué modelo obteiene mejor F1.

¿Qué modelo obteine mejor macro F1?, ¿cuál es más rápido?

**Nota**: simplemente tienes que modificar el nombre del modelo a cargar en la clase AutoTokenizer y la clase AutoModelForSequenceClassification. También es recomendable que pruebes con distintas configuraciones de hiperparámetros.
